In [ ]:
from google.colab import drive

drive.mount('/content/drive')

import os
import random
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter

# --- ROOT FOLDERS & CONFIG ---
file_path = "/content/drive/MyDrive/BIRDS_DATASET2"   # root with class folders
output_path = "/content/drive/MyDrive/Birds_Dataset3" # root for augmented class folders
target_count_per_class = 500                         # total images per class

os.makedirs(output_path, exist_ok=True)
random.seed(42)
np.random.seed(42)


def random_augment(img):
    """Apply a random sequence of augmentations to a PIL Image (RGB)."""
    w, h = img.size

    # Horizontal flip
    if random.random() < 0.5:
        img = img.transpose(Image.FLIP_LEFT_RIGHT)

    # Small rotation
    if random.random() < 0.5:
        angle = random.uniform(-20, 20)  # degrees
        img = img.rotate(angle, resample=Image.BICUBIC)

    # Random crop & resize back (acts like zoom/translation)
    if random.random() < 0.7:
        scale = random.uniform(0.7, 1.0)  # 0.7 = tighter crop, 1.0 = no crop
        new_w, new_h = int(w * scale), int(h * scale)
        if new_w < w and new_h < h:
            left = random.randint(0, w - new_w)
            top = random.randint(0, h - new_h)
            img = img.crop((left, top, left + new_w, top + new_h))
            img = img.resize((w, h), Image.LANCZOS)

    # Brightness
    if random.random() < 0.7:
        enhancer = ImageEnhance.Brightness(img)
        img = enhancer.enhance(random.uniform(0.7, 1.3))

    # Contrast
    if random.random() < 0.7:
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(random.uniform(0.7, 1.3))

    # Colour / saturation
    if random.random() < 0.7:
        enhancer = ImageEnhance.Color(img)
        img = enhancer.enhance(random.uniform(0.7, 1.3))

    # Slight blur
    if random.random() < 0.3:
        radius = random.uniform(0.3, 1.0)
        img = img.filter(ImageFilter.GaussianBlur(radius=radius))

    # Add Gaussian noise
    if random.random() < 0.5:
        arr = np.asarray(img).astype("float32")
        noise = np.random.normal(0, 8, arr.shape)  # mean 0, std 8
        arr = arr + noise
        arr = np.clip(arr, 0, 255).astype("uint8")
        img = Image.fromarray(arr)

    return img


def process_class_dir(input_dir, output_dir, target_count):
    """Copy originals and create augmentations for one class folder."""
    image_paths = [
        os.path.join(input_dir, f)
        for f in os.listdir(input_dir)
        if f.lower().endswith((".png", ".jpg", ".jpeg"))
    ]

    if not image_paths:
        print(f"No images found in '{input_dir}', skipping.")
        return

    os.makedirs(output_dir, exist_ok=True)

    class_name = os.path.basename(input_dir)
    print(f"\nProcessing class '{class_name}'")
    print(f"Found {len(image_paths)} base images in '{input_dir}'.")

    idx = 0

    # 1) Copy originals
    for path in image_paths:
        img = Image.open(path).convert("RGB")
        idx += 1
        out_path = os.path.join(output_dir, f"image_{idx:04d}.png")
        img.save(out_path)
    print(f"Copied {idx} original images to '{output_dir}'.")

    # 2) Generate augmentations
    while idx < target_count:
        src_path = random.choice(image_paths)
        img = Image.open(src_path).convert("RGB")
        aug = random_augment(img)

        idx += 1
        out_path = os.path.join(output_dir, f"image_{idx:04d}.png")
        aug.save(out_path)

        if idx % 100 == 0 or idx == target_count:
            print(f"[{class_name}] Created {idx} images so far...")

    print(f"Done '{class_name}'. Saved {idx} images total in '{output_dir}'.")


# --- MAIN: LOOP OVER ALL CLASS FOLDERS ---
class_folders = [
    d for d in sorted(os.listdir(file_path))
    if os.path.isdir(os.path.join(file_path, d))
]

if not class_folders:
    raise RuntimeError(f"No subfolders found in '{file_path}'")

print(f"Found {len(class_folders)} class folders in '{file_path}'.")

for folder_name in class_folders:
    in_dir = os.path.join(file_path, folder_name)
    out_dir = os.path.join(output_path, folder_name)
    process_class_dir(in_dir, out_dir, target_count_per_class)

print("\nAll class folders processed.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 49 class folders in '/content/drive/MyDrive/BIRDS_DATASET2'.

Processing class 'Aythya australi_300x300'
Found 140 base images in '/content/drive/MyDrive/BIRDS_DATASET2/Aythya australi_300x300'.
Copied 140 original images to '/content/drive/MyDrive/Birds_Dataset3/Aythya australi_300x300'.
[Aythya australi_300x300] Created 200 images so far...
[Aythya australi_300x300] Created 300 images so far...
[Aythya australi_300x300] Created 400 images so far...
[Aythya australi_300x300] Created 500 images so far...
Done 'Aythya australi_300x300'. Saved 500 images total in '/content/drive/MyDrive/Birds_Dataset3/Aythya australi_300x300'.

Processing class 'Burhinus grallarius_300x300'
Found 293 base images in '/content/drive/MyDrive/BIRDS_DATASET2/Burhinus grallarius_300x300'.
Copied 293 original images to '/content/drive/MyDrive/Birds_Dataset3/Burhinus grallarius_3